## Configuring the workflow

RNA-Seq-Pop, like all snakemake workflows, should be configured with a configuration file (config.yaml).

The configuration file looks like the following:

```
metadata: config/samples.tsv                    # samplesheet metadata file

dataset: 'Ag_Bouake'                            # Dataset name: Can be anything, will be used to name some main output files

## Fastq filenames and locations can either be specified in the config/samples.tsv file in columns called
## fq1 and fq2 (paired end reads) OR if auto: True -
## gzipped fastq files should be stored in the resources/reads/ directory
## If the sample name is Kisumu1 in the samples.tsv file, the paired-end reads 
## should be named Kisumu1_1.fastq.gz, and Kisumu1_2.fastq.gz. 
## Treatment names in samples.tsv should not contain underscores
fastq:
      auto: True                                           # set auto if your files follow the above rules

```

**metadata**: path to the sample metadata file    
**dataset**: give this dataset a name    
**fastq**:auto: If true, fastq files are in resources/reads/ and following `{sampleID_1.fastq.gz}` pattern. If False,             there are fq1 and fq2 columns pointing to the reads files for each sample.

```
# Chromosome names for the appropriate species.
# Currently, the variant calling and analysis part of workflow requires genes mapped to chromosomes and their physical location.
# Please ensure that these correspond exactly to the reference fasta/gff files. Extra unwanted chromosomes (unplaced contigs) can be ignored. 
# if you need to remove a prefix (Such as "AgamP4_"or "AaegL5_") from a file, use this sed command - i.e - "sed s/AgamP4_// An.gambiae.fasta > An.gambiae_clean.fasta"
contigs: ['2L', '2R', '3L', '3R', 'X']

# Paths for reference files 
reference:
  genome:
        "resources/reference/Anopheles-gambiae-PEST_CHROMOSOMES_AgamP4.fa"        # Path to genome reference FASTA file
  transcriptome:
        "resources/reference/Anopheles-gambiae-PEST_TRANSCRIPTS_AgamP4.12.fa"     # Path to transcriptome reference FASTA file
  gff:
        "resources/reference/Anopheles-gambiae-PEST_BASEFEATURES_AgamP4.12.gff3"  # Path to GFF annotation file
  snpeffdb:
        "Anopheles_gambiae"                                                       # SNPeff database name
  genes2transcripts:                                                                
        "resources/exampleGene2TranscriptMap.tsv"                                 # gene names file with gene and transcript names

**contigs:** a list of contigs you wish to analyse, must match reference files.

reference files:   
**genome:** path to the genome (.fa)     
**transcriptome:** path to the transcriptome (.fa)  
**gff:** path to the genome feature file (.gff3)  
**genes2transcripts:** path to the gene to transcript map (.tsv)  

```
# List of contrasts to do Differential Expression and Fst analysis on. 
# Should correspond to the 'treatment' columns of the samples.tsv file, and be separated by an underscore "_". i.e 'control_case', or 'Kisumu_Tiassale'.
# The treatment names themselves must not contain underscores
contrasts:
      - 'Kisumu_gambiaeCont'
      - 'gambiaeCont_gambiaePM'

cutadapt:
      activate: False
      adaptors:

QualityControl:                               # Activate Quality control analyses
      activate: True

**contrasts**: a list of the pairwise contrasts you wish to run, from values in the treatment column of the sample metadata file. The format is `control_case`.   
**cutadapt**:     
    *activate*: if True trim reads  
    *adaptors*: adaptors to be trimmed   
**QualityControl**:    
    *activate*: If True run quality control analyses   

```
DifferentialExpression:
      activate: True                          # Activate differential expression analyses

      venn:
            padj_threshold: 0.05
            groups: ['gambiaeCont_gambiaePM', 'Kisumu_gambiaeCont']

      geneHeatmaps:
            activate: False
            gene_list_tsv: "resources/example_heatmap_gene_list.tsv"

      progressiveGenes:                       # Activate progressiveGenes analysis. Will find genes that are consistently up or downregulated across two comparisons. 
            activate: True                    # For example - which genes upregulated in Kisumu v gambiaeCont, are also upregulated in gambiaeCont v gambiaePM
            padj_threshold: 0.05
            fc_threshold: 1
            groups: "Kisumu_gambiaeCont_gambiaePM"

      GSEA:                                 # Activate fgsea Gene set enrichment analysis 
            activate: True
            gaf: "resources/reference/VectorBase-50_AgambiaePEST_GO.gaf"
            KeggSpeciesID: "aga"            # 3 letter KEGG species identifier. "aga" = Anopheles gambiae/coluzzii. 
            replaceString: "AgaP_"          # An.gambiae KEGG pathways have this string appended which needs removing

**Differential Expression**: if true run differential expression analysis 
**venn**: if true make venn diagrams of de expressed genes
**progressiveGenes**: 
**gsea**:

```
VariantAnalysis:
      activate: True
      caller: 'freebayes'
      ploidy: 10                          # Ploidy level for freebayes to call at (Generally we are using pooled samples).For diploid organisms, this should be 2 * number of individuals in each pool
      chunks: 9                           # Number of chunks to split each chromosome into when parallelising freebayes. 9 or less is recommended. 

      pca:                                # Run PCA on the genotype data?
            activate: True
            missingness: 1 
      
      summaryStatistics:                  # Estimate Population Genetic Summary Statistics such as Dxy, Pi
            activate: True
            missingness: 1
      
      selection:                          # Calculate Fst and PBS per gene and in windows
            activate: True
            missingness: 1
            
            # Population Branch Statistic analysis (Needs three conditions, two closely related and an outgroup)
            pbs:
                  activate: True          
                  contrasts: 
                        - 'gambiaePM_gambiaeCont_Kisumu'
                  windownames: ['small', 'large']
                  windowsizes: [1000, 5000]
                  windowsteps: [500, 1000]

    # For An.gambiae s.l only - run Ancestry Informative Marker analysis. AIM datasets included in git repo.
      ancestry:                                
            activate: True
            missingness: 0.5                                # proportion between 0 and 1 
            gambcolu: "resources/gamb_vs_colu.zarr"         # path to gambcolu AIMs
            arab: "resources/gambcolu_vs_arab.zarr"         # path to arab AIMs

      # For Anopheles gambiae s.l only - Run analysis with compkaryo to determine proportion % karyotype
      karyotype:                            
            activate: True
            inversions:
                  - "2La"
                  - "2Rb"
```

```
miscellaneous:
      # Creates allele count .xlsx reports on specific mutations of choice
      VariantsOfInterest:             
            activate: True                             
            path: "resources/exampleMutations.tsv"         
      
      GeneFamiliesHeatmap:
            activate: True
            eggnog: resources/Anogam_long.pep_eggnog_diamond.emapper.annotations.GO
            pfam: resources/Anogam_long.pep_Pfamscan.seqs

      # find DE genes that lie under Ag1000g selective sweeps 
      # Only applicable for Anopheles gambiae s.l
      sweeps:
            activate: True
            padj_threshold: 0.05
            fc_threshold: 1.5
```